In [0]:
import os
import io

from azure.storage.filedatalake import DataLakeServiceClient
from azure.storage.blob import BlobServiceClient

import numpy as np 
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import Lasso

import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature

In [0]:
# Replace with your details
storage_account_name = ""
storage_account_key = ""

# Connect to ADLS
service_client = DataLakeServiceClient(
    account_url=f"https://{storage_account_name}.dfs.core.windows.net",
    credential=storage_account_key,
    api_version="2023-11-03"  # Use the correct supported API version
)

# List Containers
containers = service_client.list_file_systems()
for container in containers:
    print(container.name)

amzecomdata
globalmartmarketingdata
inventorydata
optimalchannel


In [0]:

def read_csv_from_blob(storage_account_name, container_name, file_name, storage_account_key=None):
    """
    Read a CSV file from Azure Blob Storage using Python and return a Pandas DataFrame.

    :param storage_account_name: Azure storage account name.
    :param container_name: Blob container name.
    :param file_name: Name of the file in the container.
    :param storage_account_key: Storage account access key.
    :return: Pandas DataFrame.
    """
    
    if not storage_account_key:
        # Try to get the key from environment variables if not provided
        storage_account_key = os.environ.get('AZURE_STORAGE_KEY')
        
    if not storage_account_key:
        raise ValueError("Storage account key must be provided either as a parameter or as an environment variable 'AZURE_STORAGE_KEY'")
    
    try:
        # Create a connection string
        connection_string = f"DefaultEndpointsProtocol=https;AccountName={storage_account_name};AccountKey={storage_account_key};EndpointSuffix=core.windows.net"
        
        # Create the BlobServiceClient
        blob_service_client = BlobServiceClient.from_connection_string(connection_string)
        
        # Get the container client
        container_client = blob_service_client.get_container_client(container_name)
        
        # Get the blob client
        blob_client = container_client.get_blob_client(file_name)
        
        # Download the blob content
        download_stream = blob_client.download_blob()
        
        # Convert the content to a DataFrame
        content = download_stream.readall()
        df = pd.read_csv(io.BytesIO(content))
        
        return df
    
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return None

In [0]:
training_data = read_csv_from_blob(storage_account_name=storage_account_name,
                                      container_name="globalmartmarketingdata", 
                                      file_name="PreProcessing_final_data.csv",
                                      storage_account_key=storage_account_key)
training_data.head()

,Unnamed: 0,week,sales_amount,base_price,final_price,promotion_type,facebook_spend,google ads_spend,influencer marketing_spend,instagram_spend,ooh_spend,print_spend,radio_spend,tv_spend,youtube_spend,facebook_ctr,google ads_ctr,influencer marketing_ctr,instagram_ctr,youtube_ctr
0,0,2023-01-01,13516527.77,101.830513,94.148539,Percentage Discount,1152.82,810.68,1122.12,707.48,11230.47,6214.43,6723.33,11311.42,703.13,3.047174,4.333516,2.471559,2.008197,2.116972
1,1,2023-01-08,95081753.02,101.830513,94.148539,Percentage Discount,7472.53,6973.85,7179.02,6834.33,11380.75,11069.60,9505.50,14004.01,6562.00,2.636847,2.732868,2.930657,3.354279,3.570124
2,2,2023-01-15,94804406.04,101.830513,94.148539,Percentage Discount,7204.29,7383.50,7185.08,6963.40,10270.30,8861.17,7836.04,14442.24,7318.19,2.679349,3.136116,2.879586,2.938546,3.776793
3,3,2023-01-22,94833974.28,101.830513,94.148539,Percentage Discount,7726.84,6522.20,7710.31,7479.41,8335.56,11601.91,6663.51,11917.89,7745.75,3.015955,3.836348,2.983655,2.858832,2.823088
4,4,2023-01-29,94806994.45,101.830513,94.148539,Percentage Discount,6987.22,6969.68,7094.25,7294.12,9575.21,7488.18,12158.00,6753.84,7044.42,2.744554,3.511152,2.338256,2.403631,3.019390


In [0]:
# Select features for scaling and encoding
numerical_features = ['sales_amount', 'base_price', 'final_price',
                      'facebook_spend', 'google ads_spend', 'influencer marketing_spend',
                      'instagram_spend', 'ooh_spend', 'print_spend', 'radio_spend',
                      'tv_spend', 'youtube_spend', 'facebook_ctr', 'google ads_ctr',
                      'influencer marketing_ctr', 'instagram_ctr', 'youtube_ctr']

categorical_features = ['promotion_type']

In [0]:
# Step 1: Scale numerical features
scaler = StandardScaler()
scaled_numerical_data = scaler.fit_transform(training_data[numerical_features])

# Convert scaled numerical data to a DataFrame
scaled_numerical_df = pd.DataFrame(scaled_numerical_data, columns=numerical_features)

# Step 2: Encode categorical features
encoder = OneHotEncoder(sparse_output=False)
encoded_categorical_data = encoder.fit_transform(training_data[categorical_features])

# Get column names for encoded categorical features
encoded_categorical_columns = encoder.get_feature_names_out(categorical_features)

# Convert encoded categorical data to a DataFrame
encoded_categorical_df = pd.DataFrame(encoded_categorical_data, columns=encoded_categorical_columns)

# Step 3: Combine scaled numerical data and encoded categorical data
processed_df = pd.concat([scaled_numerical_df, encoded_categorical_df], axis=1)

# Display the processed DataFrame
processed_df.head()

,sales_amount,base_price,final_price,facebook_spend,google ads_spend,influencer marketing_spend,instagram_spend,ooh_spend,print_spend,radio_spend,tv_spend,youtube_spend,facebook_ctr,google ads_ctr,influencer marketing_ctr,instagram_ctr,youtube_ctr,promotion_type_Buy One Get One Free,promotion_type_Percentage Discount
0,-7.208240,-1.888843,-1.079989,-6.138594,-6.036483,-6.258867,-6.414760,0.452694,-1.329475,-0.967204,0.269455,-6.263100,0.408029,2.567653,-1.354243,-2.366897,-2.237695,0.0,1.0
1,0.156877,-1.888843,-1.079989,0.671809,0.104967,0.342237,-0.124517,0.511463,0.407651,0.102746,1.192669,-0.319184,-0.548539,-0.479803,-0.315938,0.701320,1.459348,0.0,1.0
2,0.131833,-1.888843,-1.079989,0.382741,0.513173,0.348841,0.007995,0.077207,-0.382501,-0.539284,1.342926,0.447983,-0.449456,0.287937,-0.431442,-0.246289,1.985146,0.0,1.0
3,0.134503,-1.888843,-1.079989,0.945865,-0.345091,0.921263,0.537766,-0.679400,0.598106,-0.990209,0.477397,0.881749,0.335249,1.621101,-0.196076,-0.427986,-0.441227,0.0,1.0
4,0.132067,-1.888843,-1.079989,0.148817,0.100812,0.249850,0.347535,-0.194618,-0.873741,1.122828,-1.293211,0.170239,-0.297450,1.001963,-1.655725,-1.465558,0.058197,0.0,1.0


In [0]:
# Check if there are any zero or negative values in the numerical columns
print((processed_df[numerical_features] <= 0).sum())

sales_amount                   1
base_price                    27
final_price                   27
facebook_spend                24
google ads_spend              19
influencer marketing_spend    22
instagram_spend               19
ooh_spend                     28
print_spend                   28
radio_spend                   31
tv_spend                      24
youtube_spend                 24
facebook_ctr                  24
google ads_ctr                27
influencer marketing_ctr      31
instagram_ctr                 29
youtube_ctr                   27
dtype: int64


In [0]:
# Replace zero or negative values with a small positive value (e.g., 1e-6)
processed_df[numerical_features] = processed_df[numerical_features].applymap(lambda x: max(x, 1e-6))

# Apply log transformation to the target variable (sales_amount) and features (X)
target_column = 'sales_amount'
X = processed_df.drop(columns=[target_column])
y = processed_df[target_column]

# Apply log transformation (log-log regression)
X_log = np.log1p(X)  # log(1 + x) to handle zero and negative values
y_log = np.log1p(y)  # log(1 + y) to handle zero and negative values

In [0]:
# Start MLflow run
with mlflow.start_run() as run:
    # Initialize Lasso model with a regularization parameter (alpha)
    lasso_model = Lasso(alpha=0.1)  
    
    # Train the model on log-transformed features and target
    lasso_model.fit(X_log, y_log)
    
    # Get the coefficients of the Lasso model
    lasso_coefficients = pd.DataFrame({
        'Feature': X_log.columns,
        'Coefficient': lasso_model.coef_
    }).sort_values(by='Coefficient', ascending=False)
    
    # Logging the model score 
    mlflow.log_metric("r_squared", lasso_model.score(X_log, y_log))
    
    # Logging coefficients 
    for feature, coef in zip(lasso_coefficients['Feature'], lasso_coefficients['Coefficient']):
        mlflow.log_metric(f"coef_{feature}", coef)  
    
    # Save the coefficients DataFrame as a CSV and log it as an artifact
    coefficients_file_path = "/tmp/lasso_coefficients.csv"
    lasso_coefficients.to_csv(coefficients_file_path, index=False)  # Saving DataFrame to CSV file
    
    # Log the CSV file as an artifact 
    mlflow.log_artifact(coefficients_file_path)  # Log the CSV file as an artifact
    
    # Logging the model 
    mlflow.log_param("model_type", "lasso") 
    mlflow.sklearn.log_model(lasso_model, "log_reg_model")

2025/05/09 08:08:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Uploading artifacts:   0%|          | 0/9 [00:00<?, ?it/s]

🏃 View run overjoyed-shrike-789 at: https://adb-5403682037278588.8.azuredatabricks.net/ml/experiments/1100311391393808/runs/88eaf56f92384a5ab5a589658235b7f7
🧪 View experiment at: https://adb-5403682037278588.8.azuredatabricks.net/ml/experiments/1100311391393808


In [0]:
import requests
import json

# Databricks workspace URL and API token
databricks_url = "https://<your instance>.azuredatabricks.net"
api_token = " "

# Define the path to your Python script in DBFS
# This script will contain the inference logic using the trained model
python_file_path = "dbfs:/FileStore/path/to/inference.py"  # Ensure the script is uploaded to DBFS

# Job configuration for batch inference
job_data = {
    "name": "Batch Inference Job",
    "new_cluster": {
        "spark_version": "15.4.x-cpu-ml-scala2.12",  # Databricks runtime version with ML support
        "node_type_id": "Standard_DS3_v2",  # Use the Standard_DS3_v2 cluster type
        "num_workers": 2,  # Number of worker nodes (can adjust based on your requirements)
        "driver_memory": "14g",  # 14 GB memory for the driver node
        "driver_cores": 4,  # 4 cores for the driver node
    },
    "libraries": [
        {"pypi": {"package": "mlflow"}},  # Install MLflow
        {"pypi": {"package": "scikit-learn"}},  # Install scikit-learn
    ],
    "spark_python_task": {
        "python_file": python_file_path,  # Path to the Python script that will handle batch inference
        "parameters": ["--input", "dbfs:/FileStore/path/to/input_data.csv"]  # Path to the input data for inference
    }
}

# Set headers for authentication
headers = {
    "Authorization": f"Bearer {api_token}",
    "Content-Type": "application/json"
}

# API endpoint to create a Databricks job
endpoint = f"{databricks_url}/api/2.0/jobs/create"

# Send POST request to create the job
response = requests.post(endpoint, headers=headers, data=json.dumps(job_data))

# Check if the job creation was successful
if response.status_code == 200:
    print("Job created successfully:", response.json())
else:
    print("Error creating job:", response.status_code, response.text)


Job created successfully: {'job_id': 635270909845232}


In [0]:
import requests
import json

# Databricks workspace URL and API token
databricks_url = "https://<your instance>.azuredatabricks.net"
api_token = " "

# Job ID (replace with the actual Job ID from your job creation)
job_id = "635270909845232"

# Endpoint to trigger the job
run_endpoint = f"{databricks_url}/api/2.0/jobs/run-now"

# Payload to trigger the job
run_payload = {
    "job_id": job_id
}

# Send POST request to trigger the job
run_response = requests.post(run_endpoint, headers={
    "Authorization": f"Bearer {api_token}",
    "Content-Type": "application/json"
}, data=json.dumps(run_payload))

# Check if the job run started successfully
if run_response.status_code == 200:
    print("Job run started successfully:", run_response.json())
else:
    print("Error starting job run:", run_response.status_code, run_response.text)


Job run started successfully: {'run_id': 642193759589231, 'number_in_job': 642193759589231}


After running the job, the predictions will be saved in the specified location, in this case:

* Predictions: The batch inference results will be stored in /dbfs/FileStore/outputs/predictions.csv.

You can download the predictions directly from DBFS or view them within Databricks. 

But in this case, this job is running blank and failed to do the same. 